## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
#from config import g_key
g_key = "c334a28706460f469f090fde04b34da0"

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 

# Add a variable for the files to load
city_data_to_load = os.path.join("..","Weather_Database","WeatherPy_Database.csv")

# Read the City Data
city_data_df = pd.read_csv(city_data_to_load)
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Busselton,AU,-33.6500,115.3333,52.05,64,3,11.32
1,1,Rikitea,PF,-23.1203,-134.9692,71.01,68,47,11.43
2,2,Bethel,US,41.3712,-73.4140,83.64,83,5,3.67
3,3,Hilo,US,19.7297,-155.0900,84.27,75,75,10.36
4,4,Harindanga,IN,22.0383,88.3214,84.18,84,96,6.22
5,5,Ushuaia,AR,-54.8000,-68.3000,35.26,64,75,5.75
6,6,Iqaluit,CA,63.7506,-68.5145,44.33,76,75,3.44
7,7,Yinchuan,CN,38.4681,106.2731,62.01,46,0,3.47
8,8,Punta Arenas,CL,-53.1500,-70.9167,33.91,75,75,2.30
9,9,Arraial Do Cabo,BR,-22.9661,-42.0278,71.56,83,0,18.41


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 150


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Busselton,AU,-33.6500,115.3333,52.05,64,3,11.32
1,1,Rikitea,PF,-23.1203,-134.9692,71.01,68,47,11.43
2,2,Bethel,US,41.3712,-73.4140,83.64,83,5,3.67
3,3,Hilo,US,19.7297,-155.0900,84.27,75,75,10.36
4,4,Harindanga,IN,22.0383,88.3214,84.18,84,96,6.22
7,7,Yinchuan,CN,38.4681,106.2731,62.01,46,0,3.47
9,9,Arraial Do Cabo,BR,-22.9661,-42.0278,71.56,83,0,18.41
10,10,Hermanus,ZA,-34.4187,19.2345,54.90,83,52,6.58
11,11,Severo-Kurilsk,RU,50.6789,156.1250,56.03,90,79,15.28
12,12,San Matias,BO,-16.3667,-58.4000,86.83,23,0,6.08


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       596
City          596
Country       593
Lat           596
Lng           596
Max Temp      596
Humidity      596
Cloudiness    596
Wind Speed    596
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.count()

City_ID       593
City          593
Country       593
Lat           593
Lng           593
Max Temp      593
Humidity      593
Cloudiness    593
Wind Speed    593
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
#hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current description", "Lat", "Lng"]].copy()
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Busselton,AU,52.05,-33.6500,115.3333,
1,Rikitea,PF,71.01,-23.1203,-134.9692,
2,Bethel,US,83.64,41.3712,-73.4140,
3,Hilo,US,84.27,19.7297,-155.0900,
4,Harindanga,IN,84.18,22.0383,88.3214,
7,Yinchuan,CN,62.01,38.4681,106.2731,
9,Arraial Do Cabo,BR,71.56,-22.9661,-42.0278,
10,Hermanus,ZA,54.90,-34.4187,19.2345,
11,Severo-Kurilsk,RU,56.03,50.6789,156.1250,
12,San Matias,BO,86.83,-16.3667,-58.4000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    hotels
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.
print(len(hotel_df))
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"]==""].index)
print(len(clean_hotel_df))

593
0


In [17]:
clean_hotel_df.isnull().sum()

City          0.0
Country       0.0
Max Temp      0.0
Lat           0.0
Lng           0.0
Hotel Name    0.0
dtype: float64

In [18]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current weather</dt><dd>{Current description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

TraitError: The 'locations' trait of a WeightedHeatmap instance must be of length 1 <= L <= 9223372036854775807, but a value of [] was specified.